# 00 - Introduction

What is metaprogramming?

Wikipedia: A programming technique in which computer programs have the ability to treat other programs as their data.

It means that a program can be designed to read, generate, analyse, or transform other programs, and even modify itself while running.

The basic idea is we can use **code** to modify **code**.

Here are some examples:

- decorators: use a function to modify the behaviour of another function (typically).
- descriptors: use code to modify the behaviour of the dot (`.`) operator.
- metaclasses:
    - use code to create classes (types; same thing). Can be though of class (type) factory.
    - Superficially, metaclasses are not difficult to understand, but the details can get complicated.
    - Knowing when to use them is not easy. Generally speaking, they're useful for libraries/frameworks, not for application code.
    - This section will take lots of time to understand and absorb - re-reading will probably be necessary.

# 01 - Decorators and Descriptors - Review

Covered extensively in other summaries - no need to repeat here.

# 02 - The `__new__` Method

We've studied the `__init__` method quite a bit so far. It is basically a method that gets called right after the class instance has been created, usually invoked when we call the Class with arguments to instantiate an instance.

The `__new__` method is the method that is invoked to actually create the new object, as an instance of the desired class.

Since the `object` class provides a default implementation for `__new__` we rarely have to bother with it, but sometimes we want to intercept the instance creation to tweak things a bit.

The `__new__` method, unlike the `__init__` method is actually a **static** method, not an **instance** method. Which kinds of make sense since the instance does not exist yet - that's what the `__new__` method is trying to create.

Why it's not a **class** method is more complicated. We'll see why that's the case as we explore `__new__`.

Remember how we create instances of a class - we call the class with whatever arguments we need to initialize the class state:

```python
p = Person(name, age)
```

The creation of the class instance is then done in two steps:
1. The `__new__` method is called via inheritance from `object`. It receives, as arguments, the class object we want an instance of, and any additional arguments we pass to the creation call (e.g. `name` and `age`). It should return a new instance of the class (and it may have used the arguments to initialize stuff in the class too, that's up to how you write your `__new__` method)
3. If the object returned by `__new__` is an instance of the class specified in the call to `__new__`, then the `__init__` method is also called. The `__init__` method is an instance method and does not return anything (well, it returns None).

The manual approach to creating an instance of the class above is as follows:
```python
p = object.__new__(Person)
p.__init__(name, age)
```
There are some small differences between these two approaches which we'll cover soon.

The `__new__` method has the following parameters: `object.__new__(class, *args, **kwargs)`
- The `*args` and `**kwargs` are ignored, **but**, they must match what we intend to pass to the `__init__`of `class`
- A new object of type `class` is returned

##### Overriding the `__new__` method

We *can* override this method in our own custom classes. 

This should return a new object which is an instance of `class` but it doesn't *have to*, the consequence being that `__init__` is no longer called on the instance of `class`.

We don't typically override the internal implementations of `__new__`; rather, we do something before/after creating the object which we perform by delegating to `super().__new__` - similar to how we use decorators. 

We typically move all of the `__init__` logic into `__new__` after the instance has been instantiated because that's no different to running the logic in `__init__`.

So, **in metaprogramming, we should only implement one of the two**.

Here's how we can do it in practice:

In [6]:
class Person:
    def __new__(cls, name, age):
        # do stuff here
        instance = super().__new__(cls)  
        # do stuff here
        return instance

When we call `Person('Guido', 68)`:
- Python calls `__new__(Person, 'Guido', 68)`
- `__new__` returns an object
    - if that object is of the same type as the "requested":
        - `new_object.__init__('Guido', 68)` is called
    - else:
        - don't attempt to call the `__init__`.

Note: 
- We don't need to pass the other *args/**kwargs to the `super().__new__()`, just the `Person.__new__()`. If we define an `__init__` for this `Person` class, we must have `name` and `age` as parameters.
- Using `object.__new__` instead of `super().__new__` wil raise issues when dealing with inheritance; child classes will be instantiated and initialised but parent classes will only be initialised, **not instantiated**.

#### Code

When we want to inherit from the builtin types such as `int`, `float`, etc., we can only override the `__new__`, not the `__init__` due to the implementation details in the C language.

So, if we wanted to create an integer using the `int()` constructor, but instead of just returning the integer, we return the square of it, we may try: 

In [3]:
class Squared(int):
    def __init__(self, x):
        super().__init__(x ** 2)

res = Squared(4)

TypeError: object.__init__() takes exactly one argument (the instance to initialize)

But the correct approach is:

In [5]:
class Squared(int):
    def __new__(cls, x):
        return super().__new__(cls, x ** 2)

Squared(4)

16

Here's a quick example:

Let's say we want to write a bound method that calculates the area. One (unconventional) way we could do this is: 

In [17]:
class Square:
    area = lambda self: self.w * self.l

    def __init__(self, w, l):
        self.w = w
        self.l = l

s = Square(3, 4)

s.area()

12

`area` is an honest-to-goodness bound method of `s`, which means the first argument is the instance `self`.

In [25]:
s.area

<bound method Square.<lambda> of <__main__.Square object at 0x000001C0B8207640>>

Another way we can do this is via `__new__`:

In [27]:
class Square:
    def __new__(cls, w, l):
        cls.area = lambda self: self.w * self.l
        instance = super().__new__(cls)
        instance.w = w
        instance.l = l
        return instance

s = Square(3, 4)

s.area()

12

# 03 - How Classes are Created

Classes are instances of `type`, so we often say classes *are* types, and types are callable. Like all other objects in Python, `type` inherits from `object`.

#### Inner mechanics of class creation

Let's take the following code as an example:
```python
class Person:
    planet = 'Earth'
    name = property(fget=lambda self: self._name)

    def __init__(self, name):
    self._name = name
```

Here are the following steps of class creation:

1. The class **body** is extracted (and treated as a blob of text for now).
2. A new dictionary is created - this will be the namespace of the new class.
3. The body code is executed which populates the namespace, adding symbols (keys) for `planet`, `name`, `__init__`. (If this is confusing, just imagine this body of text in the global (module) namespace. The module namespace will have these exact symbols.)
4. Python uses the `type` class to create a new instance of `type` using the name of the class `Person`, the classes it inherits from, and the namespace made previously. -> `type(class_name, class_bases, class_dict)`.
5. Just as calling `Person('Alex')` returns an instance of `Person`, calling `type('Person', ...)` returns an instance of `type`.

To get an idea of how this works, we can visualise the process in code with the `exec(object, globals, locals)` function:

Passing `globals()` into `globals` lets the function see our global scope whereas `locals` is the dictionary to be used to store all the symbols for the executed program.

In [2]:
namespace = {}
exec('''
a = 20
b = 20
def __init__(self):
    pass
''', globals(), namespace)

In [3]:
namespace

{'a': 20, 'b': 20, '__init__': <function __main__.__init__(self)>}

As you can see, our namespace got populated with our symbols. This is exactly what happens with our class **body**

##### Manual Class Creation

The `type` function has two primary ways of being called:
- with a single argument -> returning the type of that argument
- with three arguments (name, bases, dict) -> returning a new type

The second approach is what we're exploring. 

In [11]:
class_name = 'Circle'

Let's deal with populating a namespace using the class body first.

In [12]:
class_body = """
def __init__(self, x, y, r):
    self.x = x
    self.y = y
    self.r = r

def area(self):
    return math.pi * self.r ** 2
"""

In [13]:
class_bases = ()  # defaults to object

In [14]:
class_dict = {}

In [15]:
exec(class_body, globals(), class_dict)

In [16]:
class_dict

{'__init__': <function __main__.__init__(self, x, y, r)>,
 'area': <function __main__.area(self)>}

Now let's construct an instance of `type` with name `'Circle'`:

In [17]:
Circle = type(class_name, class_bases, class_dict)

In [20]:
Circle.__dict__

mappingproxy({'__init__': <function __main__.__init__(self, x, y, r)>,
              'area': <function __main__.area(self)>,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Circle' objects>,
              '__weakref__': <attribute '__weakref__' of 'Circle' objects>,
              '__doc__': None})

As you can see the `Circle` namespace dict contains our functions `__init__` and `area`.

In [19]:
c = Circle(0, 0, 1)

In [20]:
c.x, c.y, c.r

(0, 0, 1)

In [21]:
c.area()

3.141592653589793

So as you can see, we use the `type` class to construct new types (classes), basically creating instances of `type`.

This is why we refer to `type` as a **metaclass**. It is a class used to construct classes.

# 04 - Inheriting from type

Since `type` inherits from object, it has `__new__` and `__init__`. Calling `type` will call `__new__`. So..

`type(name, bases, class_dict)`

is similar

`type.__new__(type, name, bases, class_dict)`

which returns an instance of `type`. Remember of course that the `__init__` will no longer be called - but remember **in metaprogramming, we only implement one of the two**

Since `type` is a class, it can be used as a **base class** for a custom class: `MyClass = class MyType(type)`

We can also override `__new__` where we tweak things, but **delegate** to `type` for actually creating the `type`, e.g. for creating the class `Circle`, *not the instance of `Circle`.*

Note that, despite `MyClass` being used for **creating classes**, it is also, in itself, a class. It's a class used for creating classes - a metaclass - a type.

#### Example

Recall that the first argument of `__new__` is `cls` which will refer to the class we want to create an instance of. Since, this instance will be a class used to create other classes, I will call it `mcls`, short for metaclass.

and the remaining arguments should match the arguments of the `__init__`. 

Previously, we had to rely on the `type(name, bases, dict)` for creating our class, but if we inherit from it, we can do stuff before/after the creation process which we **delegate** back to `type`. 

Since, we'll be inheriting from `type`, calling `super().__new__` will perform the delegation.

In [31]:
import math

class CustomType(type):
    def __new__(mcls, name, bases, class_dict):
        print('Customized type creation!')
        cls_obj = super().__new__(mcls, name, bases, class_dict)  # delegate to super (type in this case)
        cls_obj.circ = lambda self: 2 * math.pi * self.r  # basically attaching a function to the class
        return cls_obj

Previously, we created `Circle` using `type(name, bases, dict)`. Now, let's go through the same process to create our `Circle` class using `CustomType` instead of `type`.

In [32]:
class_body = """
def __init__(self, x, y, r):
    self.x = x
    self.y = y
    self.r = r

def area(self):
    return math.pi * self.r ** 2
"""

And we create our class dictionary by executing the above code in the context of that dictionary:

In [33]:
class_dict = {}
exec(class_body, globals(), class_dict)

Then we create the `Circle` class:

In [34]:
Circle = CustomType('Circle', (), class_dict)

Customized type creation!


Available to our class is everything in the namespace which was populated using the `class_body` string:

In [35]:
c = Circle(0, 0, 1)
c.area()

3.141592653589793

But, don't forget! We also injected a bound method called `circ()` for instances of any classes created using the metaclass `CustomType`.

We actually injected this after **delegation**, but we could just have easily added it to our `class_dict` before delegation:

`class_dict['circ'] = lambda self: 2 * math.pi * self.r`

In [36]:
c.circ()

6.283185307179586

# 05 - Metaclasses

A **metaclass** of a class is defined as the class used to create that class.

By default, Python uses the `type` metaclass for creating our classes, but this can be overrided very easily, for example: `class Person(metaclass=MyType)`.

So, by default, Python does for example: `class Person(metaclass=type)`.

Putting it all together:

In [41]:
class MyType(type):
    def __new__(mcls, name, bases, cls_dict):
        print(f"Using custom metaclass {mcls.__name__} to create class {name}")
        # tweak things
        # delegate to `type` for class creation
        new_class = super().__new__(mcls, name, bases, cls_dict)
        # tweak some more
        # recall that __new__ should return the instance of what it promised to create
        return new_class

class Person(metaclass=MyType):
    def __init__(self, name):
        self.name = name

Using custom metaclass MyType to create class Person


This is identical to several tedious steps carried out in the previous section except Python takes care of all of this heavy lifting.

Python calls `MyType` with `MyType` itself as `mcls`, `Person` as the `name`, `object` as the `bases` (by default) and the `Person` class body as the `cls_dict`.

# 06 - Class Decorators

While metaclasses are very powerful, they can be hard to understand when reading code. Most of the time decorators work just as well while being much easier to read.

Writing class decorators is similar to writing function decorators except that the decorator needs to expect a class. Even parametrised decorators (decorator factory) are fairly intuitive: 

In [49]:
def apr(rate):
    def inner(cls):
        cls.apr = rate
        return cls
    return inner

@apr(0.02)
class SavingsAccount():
    pass

s = SavingsAccount()
s.apr

0.02

This is equivalent to:

In [50]:
class SavingsAccount():
    pass

SavingsAccount = apr(0.02)(SavingsAccount)

s = SavingsAccount()
s.apr

0.02

A metaclass can be used but we currently don't know how to pass the `rate` parameter via a metaclass approach and we'll also have inheritance issues.

So for this purpose, metaclasses are overkill! 

#### Example 1: Logger for all methods of a Class

Let's say we have a class with a number of methods. Each method takes a number of args and returns something. What if we wanted to log each call with their result?

We could decorate each method with a logger but that can become repetitive. Another option is to view the `vars` of the class and decorate anything that is a callable.

In [38]:
from functools import wraps

def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner

In [39]:
def class_logger(cls):
    for method_name, obj in vars(cls).items():
        if callable(obj):
            print("decorating: ", cls, obj)
            setattr(cls, method_name, func_logger(obj))
    return cls

So now we could do this:

In [40]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'

decorating:  <class '__main__.Person'> <function Person.__init__ at 0x0000018858A6C3A0>
decorating:  <class '__main__.Person'> <function Person.greet at 0x0000018858A6FB50>


In [41]:
p = Person('John', 78)

log: Person.__init__((<__main__.Person object at 0x00000188576D3790>, 'John', 78), {}) = None


In [42]:
p.greet()

log: Person.greet((<__main__.Person object at 0x00000188576D3790>,), {}) = Hello, my name is John and I am 78


'Hello, my name is John and I am 78'

We will run into issues with static methods and class methods as they are **not** callables. (From Python 3.10, they are callables as they implement `__call__`, in addition to `__get__`.)


They are instead **descriptors** which are classes. The consequence of this is that if we want to decorate a method with `@staticmethod` or `@classmethod` in addition to a function decorator e.g. `@func_logger`, **we must apply the `@func_logger` first and __then__ the `@staticmethod`/`@classmethod`**

If however, we want to decorate our class as before but have it figure out how to deal with static and class methods, we can do that too.

Remember that all methods are **non-data descriptors** which means:
- They have the `__get__` method.
- Depending on how `__get__` is called, it either returns the function itself or the bound method. This is how python differentiates between the two.

We know that `__get__` takes 3 parameters. Here's what they are for regular functions:
- `self`: the function itself e.g. `add` because we have `add.__get__()`.
- `instance`: either `None` or our `__main__` module; we can think of our `__main__` module as the class housing this function, so calling it directly is like calling it from the module 'class' instead of an instance.
- `owner_class`: in this case, it will be our `__main__`module

If `instance=None`, `__get__` returns the function as opposed to a bound method. Where is the pointer to that function? In the `__func__` attribute of the descriptor.

In [43]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
            
        elif isinstance(obj, staticmethod):
            original_func = obj.__func__
            print('decorating static method', original_func)
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            print(method, type(method))
            setattr(cls, name, method)
            
        elif isinstance(obj, classmethod):
            original_func = obj.__func__
            print('decorating class method', original_func)
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
            
    return cls

In [44]:
@class_logger
class Person:
    @staticmethod
    def static_method(a, b):
        print('static_method called...', a, b)
        
    @classmethod
    def class_method(cls, a, b):
        print('class_method called...', cls, a, b)
        
    def instance_method(self, a, b):
        print('instance_method called...', self, a, b)

decorating: <class '__main__.Person'> static_method
decorating class method <function Person.class_method at 0x0000018858A6E560>
decorating: <class '__main__.Person'> instance_method


In [45]:
Person.static_method(10, 20)

static_method called... 10 20
log: Person.static_method((10, 20), {}) = None


In [46]:
Person.class_method(10, 20)

class_method called... <class '__main__.Person'> 10 20
log: Person.class_method((<class '__main__.Person'>, 10, 20), {}) = None


In [47]:
Person().instance_method(10, 20)

instance_method called... <__main__.Person object at 0x00000188576D6A10> 10 20
log: Person.instance_method((<__main__.Person object at 0x00000188576D6A10>, 10, 20), {}) = None


This is good, but what about **properties**?

As you may remember, the property class has an `fset`, `fget` and `fdel` attribute which are handles to their respective functions. The issue is that these are all read-only attributes.

But, remember that properties also have `getter()`, `setter()` and `deleter()` methods which take a function, stores it in the descriptor and then returns the descriptor itself. (Remember, this is why we can only call `@<some_prop>.setter` after applying the `@property` decorator. 

In [48]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
            
        elif isinstance(obj, staticmethod):
            original_func = obj.__func__
            print('decorating static method', original_func)
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            print(method, type(method))
            setattr(cls, name, method)
            
        elif isinstance(obj, classmethod):
            original_func = obj.__func__
            print('decorating class method', original_func)
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
            
        elif isinstance(obj, property):
            print('decorating property', obj)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
    return cls

So, here we've checked to see if the property which is a descriptor has received `fget`. If it has, we want to replace the entire property with a new property with a modified `fget`. This modification involves taking the original `fget` function from our property, decorate it with `func_logger` and essentially creating a new entire property whose `fget` is equal to our decorated function.

Again, recall that the `getter()` of a property descriptor takes a function, stores it in the `fget` attribute of the descriptor, and returns a descriptor new descriptor. When we call the propery e.g. `Person.age`, the `__get__` is called on the `age` descriptor which calls the function stored in `fget`. 

In [49]:
class A:
    @property
    def x():
        print('original x called')

print(A.x)
A.x.fget()

original x called


In [50]:
def new_x():
    print('new x called')

A.x = A.x.getter(new_x)
print(A.x)
A.x.fget()

new x called


Let's now demonstrate it:

In [52]:
@class_logger
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name

decorating: <class '__main__.Person'> __init__
decorating property <property object at 0x0000018858AACF90>


In [53]:
p = Person('David')

log: Person.__init__((<__main__.Person object at 0x00000188576F8F10>, 'David'), {}) = None


In [54]:
p.name

log: Person.name((<__main__.Person object at 0x00000188576F8F10>,), {}) = David


'David'

Perfect!

But there's a problem...

We try to catch all functions in our class using the `callable` predicate function, but as we'll now show, **not all callables are functions**.

In [55]:
@class_logger
class Person:
    class Other:
        def __call__(self):
            print('called instance of Other...')
            
    other = Other()

decorating: <class '__main__.Person'> Other
decorating: <class '__main__.Person'> other


So, as you see it decorated both the class `Other` (since classes are callables), and it decorated `other` since we made instances of `Other` callable too.

So maybe we can use the `inspect` module to restrict our callables further.

In the original notebook, you can find the code to build up the following table that outlines how each predicate function in the `inspect` module treats static methods, class methods, instance methods, properties (`name`), dunder methods, classes within classes and instances of those (which have implemented `__call__`).

```
                   static_method	cls_method   	inst_method  	name         	__add__      	Other        	other        
isroutine          True         	True         	True         	False        	True         	False        	False        
ismethod           False        	False        	False        	False        	False        	False        	False        
isfunction         False        	False        	True         	False        	True         	False        	False        
isbuiltin          False        	False        	False        	False        	False        	False        	False        
ismethoddescriptor True         	True         	False        	False        	False        	False        	False   

The TLDR is that we want to use `isroutine` so that we can also decorate dunder methods, but we will still need our current implementation for properties, static methods and class methods. With some additional refactoring, we get:

In [57]:
import inspect

def class_logger(cls):
    for name, obj in vars(cls).items():
        if isinstance(obj, (staticmethod, classmethod)):
            type_ = type(obj)
            original_func = obj.__func__
            print(f'decorating {type_.__name__} method', original_func)
            decorated_func = func_logger(original_func)
            method = type_(decorated_func)
            setattr(cls, name, method)
            
        elif isinstance(obj, property):
            print('decorating property', obj)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
            
        elif inspect.isroutine(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

In [58]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        print('static_method called...')
    
    @classmethod
    def cls_method(cls):
        print('class method called...')
    
    def inst_method(self):
        print('instance method called...')
    
    @property
    def name(self):
        print('name getter called...')
    
    def __add__(self, other):
        print('__add__ called...')
    
    @class_logger
    class Other:
        def __call__(self):
            print(f'{self}.__call__ called...')
        
    other = Other()

decorating: <class '__main__.MyClass.Other'> __call__
decorating staticmethod method <function MyClass.static_method at 0x0000018858B08820>
decorating classmethod method <function MyClass.cls_method at 0x0000018858B088B0>
decorating: <class '__main__.MyClass'> inst_method
decorating property <property object at 0x0000018858ABEFC0>
decorating: <class '__main__.MyClass'> __add__


# 07 - Decorator Classes

First off, don't confuse this with class decorators - here I'm talking about using a class to create decorators - that can be used to decorate functions, or classes - but instead of the decorator being a function, it is a class whose instances will act as decorators.

We do this by making instances of the decorator class **callable**, by implementing the `__call__` method.

Here's a basic example to illustrate it:

In [1]:
class Logger:
    def __init__(self, fn):
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        print(f'Log: {self.fn.__name__} called.')
        return self.fn(*args, **kwargs)

In [2]:
@Logger
def say_hello():
    pass

In [3]:
say_hello()

Log: say_hello called.


**One limitation** with this approach is that we cannot decorate bound methods. Let's try it to see why we can't.

In [4]:
class Person:
    def __init__(self, name):
        self.name = name
        
    @Logger
    def say_hello(self):
        return f'{self.name} says hello!'

In [5]:
p = Person('David')

In [6]:
p.say_hello()

Log: say_hello called.


TypeError: Person.say_hello() missing 1 required positional argument: 'self'

We must first remember the difference between functions and methods.

Functions implement the **(non-data) descriptor protocol**: Implement the `__get__` method. If called from an instance, return a bound method. Otherwise, return the descriptor itself.

In [9]:
def my_func():
    pass
    
print(my_func)
hasattr(my_func, '__get__')

<function my_func at 0x000001C6C7D39F30>


True

In [11]:
class A:
    def my_func(self):
        pass

a = A()

print(A.my_func)  # returns the descriptor
print(a.my_func)  # returns a bound method using types.MethodType
hasattr(A.my_func, '__get__')

<function A.my_func at 0x000001C6C7F6D3F0>
<bound method A.my_func of <__main__.A object at 0x000001C6C908B940>>


True

Bound methods are created with `types.MethodType`

Here's a trimmed snippet on `MethodType` from `help`:
```
class method(object)
 |  method(function, instance)
 |  
 |  Create a bound instance method object.
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __func__
 |      the function (or other callable) implementing a method
 |  
 |  __self__
 |      the instance to which a method is bound
```

When creating our `MethodType`, the `function` parameter is the callable that will call our decorated function; in other words, it's our descriptor instance itself: `self`.

The `instance` is the object that the descriptor is bound to; in other words, it's the instance of the class containing the decorated method.

Let's create our modified `Logger`:

In [16]:
from types import MethodType

class Logger:
    def __init__(self, fn):
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        print(f'Log: {self.fn.__name__} called.')
        return self.fn(*args, **kwargs)
    
    def __get__(self, instance, owner_class):
        print(f'__get__ called: self={self}, instance={instance}')
        if instance is None:
            return self
        else:
            # self is callable, since it implements __call__
            return MethodType(self, instance)

In [17]:
class Person:
    def __init__(self, name):
        self.name = name
        
    @Logger
    def say_hello(self):
        return f'{self.name} says hello!'

In [18]:
p = Person('David')

In [19]:
p.say_hello

__get__ called: self=<__main__.Logger object at 0x000001C6C909D3C0>, instance=<__main__.Person object at 0x000001C6C905C9D0>


<bound method ? of <__main__.Person object at 0x000001C6C905C9D0>>

As you can see `say_hello` is now considered a bound method. And it bound the callable instance of Logger to the Person instance.

In [20]:
p.say_hello()

__get__ called: self=<__main__.Logger object at 0x000001C6C909D3C0>, instance=<__main__.Person object at 0x000001C6C905C9D0>
Log: say_hello called.


'David says hello!'

The last thing we should check is that the decorator works with class and static methods too.

Just remember that the order of the decorators is important - we need to decorate with our logger before we decorate with the static and class decorators. that way we end up decorating the decorated function (so just a plain fuinction decorator), and then making it into a class or static method.

In [23]:
class Person:
    @classmethod
    @Logger
    def cls_method(cls):
        print('class method called...')
        
    @staticmethod
    @Logger
    def static_method():
        print('static method called...')
        

In [24]:
Person.cls_method()

Log: cls_method called.
class method called...


In [25]:
Person.static_method()

Log: static_method called.
static method called...


# 08 - Metaclasses vs Class Decorators

The main advantages of class decorators over metaclasses are that you can:
- stack decorators,
- decorate classes in an inheritance chain with different decorators.

The main advantage of metaclasses over decorators is that:
- subclasses inherit parent metaclasses. In our example before with the class decorator `@class_logger`, any subclass of `Person`, such as `Student(Person)`, will not be logged. We would have to remember to decorate `Student` with `@class_logger`. We wouldn't have to do this if we used a metaclass instead.

# 09 - Metaclass Parameters

So this is the simplest form of creating metaclasses:

In [8]:
class MetaClass(type):
    def __new__(mcls, name, bases, class_dict):
        return super().__new__(mcls, name, bases, class_dict)

class MyClass(metaclass=MetaClass):
    pass

Can we add additional arguments to our `__new__`?

Yes, from Python 3.6 onwards:

In [9]:
class MetaClass(type):
    def __new__(mcls, name, bases, class_dict, arg1, arg2, arg3=None):
        return super().__new__(mcls, name, bases, class_dict)

class MyClass(metaclass=MetaClass, arg1=1, arg2=2, arg3=3):
    pass

Notice that the arguments **must** be passed as **named** arguments, except for default arguments like `arg3`. The reason why is because positional arguments are reserved for the parent objects that we inherit from.

Why might we want to pass additional arguments?

If we want to create a class with a number of class attributes that will only be determined at run time, we can pass these attributes in to the class definition. For example:

In [21]:
class AutoClassAttrib(type):
    def __new__(mcls, name, bases, cls_dict, extra_attrs=None):
        if extra_attrs:
            print('Creating class with some extra attributes: ', extra_attrs)
            # here I'm going to things directly into the cls_dict namespace
            # but could also create the class first, then add using setattr
            for attr_name, attr_value in extra_attrs:
                cls_dict[attr_name] = attr_value
        return super().__new__(mcls, name, bases, cls_dict)

In [22]:
class Account(metaclass=AutoClassAttrib, extra_attrs=[('account_type', 'Savings'), ('apr', 0.5)]):
    pass

Creating class with some extra attributes:  [('account_type', 'Savings'), ('apr', 0.5)]


In [23]:
Account.account_type, Account.apr

('Savings', 0.5)

An ever so slightly simpler way is the following:

In [35]:
class AutoClassAttrib(type):
    def __new__(mcls, name, bases, cls_dict, **extra_attrs):
        print('Creating class with some extra attributes: ', extra_attrs)
        cls_dict.update(extra_attrs)
        return super().__new__(mcls, name, bases, cls_dict)    

In [36]:
class Account(metaclass=AutoClassAttrib, account_type='Savings', apr=0.5):
    pass

Creating class with some extra attributes:  {'account_type': 'Savings', 'apr': 0.5}


In [38]:
Account.account_type, Account.apr

('Savings', 0.5)

# 10 - The `__prepare__` Method

We know that when we call `class MyClass(metaclass=MyMeta)`, the `__new__` of `MyMeta` is called with `__new__(mcls, name, bases, cls_dict)`.

We know `cls_dict` will become the namespace of our created classes, but where and how is the `cls_dict` populated?

In [17]:
class MyMeta(type):
    def __new__(mcls, name, bases, cls_dict):
        print(f"Using custom metaclass {mcls.__name__} to create class {name}")
        print(f"{cls_dict = }")
        new_class = super().__new__(mcls, name, bases, cls_dict)
        return new_class

In [18]:
class Person(metaclass=MyMeta):
    def __init__(self, name):
        self.name = name

Using custom metaclass MyMeta to create class Person
cls_dict = {'__module__': '__main__', '__qualname__': 'Person', '__init__': <function Person.__init__ at 0x0000026B46E42440>}


In fact, `__prepare__`, another static method implemented by `type`, is called before `__new__`.

`__prepare__` also receives any additional named args passed to our metaclass (`class MyMeta(metaclass=MyMeta, <kwargs>)`) and must return a **mapping type**, e.g. an **empty** `dict`.

Python then manipulates the empty dict, injecting in what we see above e.g. `__module__`, `__qualname__`, etc. before calling `__new__` with this dictionary as `cls_dict`.

Let's replicate what Python does, but instead of returning an empty dictionary, we'll add some items. This will also demonstrate that Python is injecting what I mentioned above, not `__prepare__`.

In [22]:
class MyMeta(type):
    @staticmethod
    def __prepare__(name, bases, **kwargs):
        print('__prepare__ called...')
        print('\tname:', name)
        print('\tkwargs:', kwargs)
        return {'a': 100, 'b': 200}

    def __new__(mcls, name, bases, cls_dict, **kwargs):
        print('__new__ called...')
        print('\tcls_dict:', cls_dict)
        print('\tkwargs:', kwargs)
        return super().__new__(mcls, name, bases, cls_dict)

In [23]:
class MyClass(metaclass=MyMeta):
    pass

__prepare__ called...
	name: MyClass
	kwargs: {}
__new__ called...
	cls_dict: {'a': 100, 'b': 200, '__module__': '__main__', '__qualname__': 'MyClass'}
	kwargs: {}


Here's an easier way of seeing what `__prepare__` returns:

In [24]:
type.__prepare__()

{}

This gives us an alternative approach to populating our `cls_dict` in `__new__`. Previously we would do:

In [25]:
class MyMeta(type):
    def __new__(mcls, name, bases, cls_dict, **kwargs):
        cls_dict.update(kwargs)
        return super().__new__(mcls, name, bases, cls_dict)

class MyClass(metaclass=MyMeta, arg1=100, arg2=200):
    pass

vars(MyClass)

mappingproxy({'__module__': '__main__',
              'arg1': 100,
              'arg2': 200,
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

But now we can do the following:

In [28]:
class MyMeta(type):
    def __prepare__(name, bases, **kwargs):
        return kwargs
        
    def __new__(mcls, name, bases, cls_dict, **kwargs):
        return super().__new__(mcls, name, bases, cls_dict)

class MyClass(metaclass=MyMeta, arg1=100, arg2=200):
    pass

vars(MyClass)

mappingproxy({'arg1': 100,
              'arg2': 200,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

while ensuring `__new__` is defined because this lets us specify additional `kwargs`. By default, `__new__` does not accept additional arguments. 

#### Classes, Metaclasses, and `__call__`

Consider the following:

In [29]:
class Person:
    def __call__(self):
        pass

We know that this makes **instances** of this class a callable, not the class itself:

In [31]:
p = Person()  # not this
p()  # but this

But, since `Person` is also callable, it must implement `__call__`. Therefore, its metaclass, `type`, implements `__call__`.

Let's chronologically lay out the steps taken when `p = Person()` is called.

1. Firstly, `type.__call__` is called.
2. `type.__call__` then calls `Person.__new__` to initialise the class, returning an instance of `Person`.
3. `type.__call__` then calls `__init__` on the instance to instantiate it. This is then associated to the symbol `p`.

Okay, but we can also do:
```python
type(name, bases, cls_dict)
```

Therefore `type` is *also* a callable, which means *its* metaclass must implement `__call__`. 

What is the metaclass of `type`? Itself.

**The type of `type` is `type`.**

So what are the chronological steps when `type` is called - which occurs when `Person = type(name, bases, cls_dict)` is called, for example.

Pretty much the exact same steps as above.

1. Firstly, `type.__call__` is called.
2. `type.__call__` then calls `type.__new__` to initialise the class, returning an instance of `type` which is the `Person` **class**.
3. `type.__call__` then calls `__init__` on the instance (`Person` is the instance) -> `Person.__init__(). This is then associated to the symbol `Person`.

Let's compare and summarise by taking a look at the chronology of traditional **declarative** class creation.

When we type `class Person(metaclass=type)`:

1. `type.__prepare__` is called returning an empty dictionary.
2. An internal python process runs to populate the dictionary with `__qualname__`, `__docs__`, etc, prepares the class body, the bases, etc., before finally calling `type.__call__`.
3. `type.__call__` calls `type.__new__` to initialise the class, returning an instance of `type` which is the `Person` **class**.
4. `type.__call__` then calls `__init__` on the instance (`Person` is the instance) -> `Person.__init__()`. This is then associated to the symbol `Person`.

# 11 - Metaprogramming Application 1

# 12 - Metaprogramming - Application 2

# 13 - Metaprogramming - Application 3

# 14 - Attribute Read Accessors

# 15 - Attribute Write Accessors

# 16 - Accessors - Application